## Import packages

In [145]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\src.py'>

## Configurations

In [94]:
path_to_extension = r'C:\Users\Matth\Desktop\3.12_0'
character_name = "Mercychan"
character_server = "Benediction"
character_region = "US"

clear_char_data = False

## Navigate to character page

In [95]:
char_url, browser = src.load_environment(path_to_extension, character_name, character_server, character_region)

In [105]:
bosses = src.get_phase3_bosses(browser)

In [110]:
# Feel free to omit bosses that you are not interested in
bosses = ["Teron Gorefiend"]

In [97]:
boss_link_dict = {"High Warlord Naj'entus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622"}

In [98]:
char_info = src.get_character_info(char_url)
char_info

'Mercychan (Benediction US)'

## Main code loop

In [146]:
def main():
    if clear_char_data:
        src.clear_character_data()
    
    for i, boss in enumerate(bosses):
        
        player_df = pd.DataFrame(pd.np.empty((0, 16)))
        player_df.columns = ["Name", "Server", "Date", "Kill time", "Rank", "nHealers", "Spriest?", "Innervate?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotations"]

        browser.get(f"https://classic.warcraftlogs.com/character/{character_region.lower()}/{character_server.lower()}/{character_name.lower()}{boss_link_dict[boss]}")
        time.sleep(1)

        # Pick out the top parse of this boss
        search = browser.find_elements_by_class_name("character-table-link")
        boss_page_url = browser.current_url
        
        print(f"Boss: {boss}")
        
        #for i in range(len(search)):
        for i in range(3):
                        
            try:
                rank, date, boss_rank, boss_HPS, boss_killtime = src.get_boss_data(browser, i)

            except IndexError: continue
                
            search = browser.find_elements_by_class_name("character-table-link")
                
            if src.check_if_parse_already_recorded(i, browser, search, boss, character_name, character_server, character_region): 
                print("Parse already recorded. Skipping")
                browser.get(boss_page_url)
                continue
            
            rank.click()
            time.sleep(2)
            
            temp_url = browser.current_url
                        
            boss_tanks = src.get_tanks(browser)
            nHealers = src.get_nHealers(browser)
            
            print(f"Tanks: {boss_tanks}")

            browser.get(temp_url)
            time.sleep(0.5)

            src.scroll_click(browser.find_element_by_link_text(character_name))
            time.sleep(0.5)
            
            LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, boss_HPS)
            time.sleep(2)
                
            spriest = src.check_spriest(browser)
            innervate = src.check_innervate(browser)

            # Download the cast-sequence and damage-taken CSVs.
            src.download_csv(browser, temp_url, "filter-casts-tab", "cast_sequence/casts.csv")
            src.download_csv(browser, temp_url, "filter-damage-taken-tab", "damage_taken/dmg_taken.csv")

            #nHealers = src.get_nHealers(browser)
            browser.get(temp_url)
            time.sleep(1)

            # For most single tank bosses, the main tank is the current boss' melee target
            if boss in ["Rage Winterchill", "Kaz'rogal", "Archimonde", "High Warlord Naj'entus", "Supremus", 
                        "Teron Gorefiend", "Gurtogg Bloodboil", "Mother Shahraz"]:

                # Clean the csvs
                df = src.clean_dmg_taken_csv(boss)
                df2 = src.clean_cast_sequence_csv()

                # Fix regrowth timer
                df2 = src.fix_cast_time(df2)

                # Add boss target
                df = src.add_boss_target(boss, df, df2)
                df = src.fix_boss_target(df)

                # Get the rotations
                try: rotations = src.calculate_rotations(df, boss, boss_tanks)
                except ZeroDivisionError:
                    print("Click element error. Skipping this parse.")
                    browser.get(boss_page_url)
                    time.sleep(0.5)
                    continue

            elif boss in ["Anetheron", "Azgalor", "Reliquary of Souls",
                         "Shade of Akama", "The Illidari Council", "Illidan Stormrage"]:

                # Clean the csv
                df = src.clean_cast_sequence_csv()
                df = src.fix_cast_time(df)

                # Get the rotations
                try: rotations = src.calculate_rotations(df, boss, boss_tanks)
                except ZeroDivisionError:
                    print("Click element error. Skipping this parse.")
                    browser.get(boss_page_url)
                    time.sleep(0.5)
                    continue

            print(f'Rotations: {rotations}')    

            # Save data to csv
            to_append = [character_name, character_server + " " + character_region, date, boss_killtime, boss_rank, str(nHealers), spriest, innervate, LB_uptime, boss_HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotations]
            series = pd.Series(to_append, index = player_df.columns)
            player_df = player_df.append(series, ignore_index=True)

            #Return to boss page
            browser.get(boss_page_url)
            time.sleep(0.5)
            print('------')
                
        # Return to character page
        browser.get(char_url)
        time.sleep(0.5)

        player_df.to_csv(f"character_data/{boss.replace(' ', '')}_{character_name}.csv", index=None)
        
        src.combine_character_data(character_name, character_server, character_region, date, boss_rank, boss)
        src.add_row_to_xlsx(boss, character_name)
        src.sort_excel(boss)
        print(f"{boss} scraping complete.")
        print("---------------")
    
        #for boss in bosses:
        os.remove(f"character_data/{boss.replace(' ', '')}_{character_name}.csv")
    

if __name__ == "__main__":
    main()

Boss: Teron Gorefiend
Parse already recorded. Skipping
Tanks: ['Hilfedanke', 'Formulatank']
Rotations: [['1LB 1I 2RG', 0.667], ['1LB 2I 1RG', 0.167]]
------
Tanks: ['Hilfedanke', 'Formulatank']
Rotations: [['1LB 2I 1RG', 0.55], ['1LB 3I', 0.25]]
------
Teron Gorefiend scraping complete.
---------------


In [119]:
from datetime import datetime

In [120]:
df = src.clean_dmg_taken_csv(boss)
df2 = src.clean_cast_sequence_csv()

In [130]:
df2.tail(50)

,Time,Type,Ability,Minute,Second,Cast Time,Target,Boss Target
51,01:20.757,Cast,Rejuvenation,1,20.757,None,Saintacy,NaN
52,01:24.129,Cast,Regrowth,1,24.129,1.92,Formulatank,NaN
53,01:26.059,Cast,Regrowth,1,26.059,1.93,Formulatank,NaN
54,01:26.059,Cast,Lifebloom,1,26.059,None,Formulatank,NaN
55,01:27.492,Cast,Rejuvenation,1,27.492,None,Fakedanaima,NaN
56,01:30.874,Cast,Regrowth,1,30.874,1.91,Formulatank,NaN
57,01:30.874,Cast,Rejuvenation,1,30.874,None,Formulatank,NaN
58,01:32.315,Cast,Lifebloom,1,32.315,None,Formulatank,NaN
59,01:35.664,Cast,Regrowth,1,35.664,1.92,Formulatank,NaN
60,01:35.678,Cast,Swiftmend,1,35.678,None,Formulatank,NaN


In [144]:
fix_cast_time(df2)

,Time,Type,Ability,Minute,Second,Cast Time,Target,Boss Target
0,00:00.944,Cast,Rejuvenation,0,0.944,None,Formulatank,NaN
1,00:02.374,Cast,Lifebloom,0,2.374,None,Formulatank,NaN
2,00:04.223,Cast,Swiftmend,0,4.223,None,Formulatank,NaN
3,00:05.691,Cast,Rejuvenation,0,5.691,None,Formulatank,NaN
4,00:07.101,Cast,Lifebloom,0,7.101,None,Formulatank,NaN
...,...,...,...,...,...,...,...,...
96,02:30.324,Cast,Lifebloom,2,30.324,None,Formulatank,NaN
97,02:31.776,Cast,Rejuvenation,2,31.776,None,Formulatank,NaN
98,02:31.274,Cast,Regrowth,2,31.274,1.94,Vladamin,NaN
99,02:33.228,Cast,Regrowth,2,33.228,1.93,Formulatank,NaN


In [143]:
def fix_cast_time(df):
    temp = []
    for i, row in df.iterrows():
        if row['Ability'] in ['Regrowth', 'Rebirth'] and row["Cast Time"] != "Canceled":
            a = str(datetime.strptime(str(row['Minute']) + ":" + str(row['Second']), "%M:%S.%f") - datetime.strptime(row['Cast Time'], "%S.%f"))
            a = a.split(".")
            if len(a) == 1: a.append('000000')
            b = a[0].split(":")
            c = b[1] + ":" + b[2] + "." + a[1][:3]
            temp.append(c)

        else:
            temp.append(row["Time"])

    df["Time"] = temp
    
    temp_df = df['Time'].str.split(":", expand=True)
    df['Second'] = temp_df[1]
    df['Second'] = pd.to_numeric(df['Second'])
    
    return df